<a href="https://colab.research.google.com/github/mohanrajmit/Human-Action-Classification-/blob/master/Action%20Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Activity Recognition

In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import h5py
from tqdm import tqdm
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model, Sequential
from keras.layers import Input, LSTM, Dense, Dropout
from keras.utils import to_categorical
from keras.applications.imagenet_utils import preprocess_input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping
from keras.utils.io_utils import HDF5Matrix

SEQ_LEN = 30
MAX_SEQ_LEN = 200
BATCH_SIZE = 16
EPOCHS = 1000

Using TensorFlow backend.


In [2]:
!git clone https://github.com/mohanrajmit/Human-Action-Classification-.git 

Cloning into 'Human-Action-Classification-'...
remote: Enumerating objects: 1269, done.
remote: Counting objects: 100% (1269/1269), done.
remote: Compressing objects: 100% (1264/1264), done.
remote: Total 1269 (delta 18), reused 1236 (delta 2), pack-reused 0
Receiving objects: 100% (1269/1269), 469.58 MiB | 30.72 MiB/s, done.
Resolving deltas: 100% (18/18), done.
Checking out files: 100% (1218/1218), done.


In [0]:
def main():
    # Get model with pretrained weights.
    base_model = InceptionV3(
    weights='imagenet',
    include_top=True)
    
    
    # We'll extract features at the final pool layer.
    model = Model(
        inputs=base_model.input,
        outputs=base_model.get_layer('avg_pool').output)
    
    # Getting the data
    df = get_data('Human-Action-Classification-/data/data_file.csv')
    
    # Clean the data
    df_clean = clean_data(df)
    
    # Creating index-label maps and inverse_maps
    label_index, index_label = get_class_dict(df_clean)
    
    # Split the dataset into train and test
    train, test = split_train_test(df_clean)
    
    # Encoding the dataset
    encode_dataset(train, model, label_index, "train")
    encode_dataset(test,model,label_index,"test")
    
   

In [4]:
!ls

Human-Action-Classification-  sample_data


In [0]:
def get_data(path, if_pd=False):
    """Load our data from file."""
    names = ['partition', 'class', 'video_name', 'frames']
    df = pd.read_csv(path,names=names)
    return df

def get_class_dict(df):
    class_name =  list(df['class'].unique())
    index = np.arange(0, len(class_name))
    label_index = dict(zip(class_name, index))
    index_label = dict(zip(index, class_name))
    return (label_index, index_label)
    
def clean_data(df):
    mask = np.logical_and(df['frames'] >= SEQ_LEN, df['frames'] <= MAX_SEQ_LEN)
    df = df[mask]
    return df
def split_train_test(df):
    partition =  (df.groupby(['partition']))
    un = df['partition'].unique()
    train = partition.get_group(un[0])
    test = partition.get_group(un[1])
    return (train, test)

def preprocess_image(img):
    img = cv2.resize(img, (299,299))
    return preprocess_input(img)
    
    
def encode_video(row, model, label_index):
    cap = cv2.VideoCapture(os.path.join("Human-Action-Classification-/data","UCF-101",str(row["class"].iloc[0]) ,str(row["video_name"].iloc[0]) + ".avi"))
    images = []  
    for i in range(SEQ_LEN):
        ret, frame = cap.read()
        frame = preprocess_image(frame)
        images.append(frame)
    
    
    features = model.predict(np.array(images))
    index = label_index[row["class"].iloc[0]]
    print(index)
    #y_onehot = to_categorical(index, len(label_index.keys()))
    
    return features, index

from keras.utils import np_utils
def encode_dataset(data, model, label_index, phase):
    input_f = []
    output_y = []
    required_classes = ["ApplyEyeMakeup" , "ApplyLipstick" , "Archery" , "BabyCrawling" , "BalanceBeam" ,
                       "BandMarching" , "BaseballPitch" , "Basketball" , "BasketballDunk"]
   
    
    for i in tqdm(range(data.shape[0])):
    # Check whether the given row , is of a class that is required
        if str(data.iloc[[i]]["class"].iloc[0]) in required_classes:
 
            features,y =  encode_video(data.iloc[[i]], model, label_index)
            input_f.append(features)
            output_y.append(y)
        
    
    le_labels = np_utils.to_categorical(output_y)
    f = h5py.File(phase+'_8'+'.h5', 'w')
    f.create_dataset(phase, data=np.array(input_f))
    f.create_dataset(phase+"_labels", data=np.array(le_labels))
    
    del input_f[:]
    del output_y[:]



In [6]:
main()

Instructions for updating:
Colocations handled automatically by placer.
96116736/96112376 [==============================] - 4s 0us/step


  0%|          | 1/703 [00:05<1:05:46,  5.62s/it]

0


  0%|          | 2/703 [00:06<47:21,  4.05s/it]  

0


...

8


100%|█████████▉| 289/290 [01:49<00:00,  2.63it/s]

8


100%|██████████| 290/290 [01:49<00:00,  2.63it/s]

8


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.layers import LSTM

In [0]:
def lstm():
    """Build a simple LSTM network. We pass the extracted features from
    our CNN to this model predominantly."""
    input_shape = (SEQ_LEN, 2048)
    # Model.
    model = Sequential()
    model.add(LSTM(2048,input_shape=input_shape,dropout=0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(9, activation='softmax'))
    #model.add(Dense(10, activation='softmax'))"""
    checkpoint = ModelCheckpoint(filepath='models\\checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5')
    
    tb_callback = TensorBoard(
    log_dir="logs",
    histogram_freq=2,
    write_graph=True
    )
    
    early_stopping = EarlyStopping(monitor = 'val_loss',patience= 10)
    
    callback_list = [checkpoint, tb_callback]

    optimizer = Adam(lr=1e-5, decay=1e-6)
    metrics = ['accuracy', 'top_k_categorical_accuracy']
    model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=metrics)
    return model, callback_list


In [9]:
x_train = HDF5Matrix('train_8.h5', 'train')
y_train = HDF5Matrix('train_8.h5', 'train_labels')
x_test = HDF5Matrix('test_8.h5', 'test')
y_test = HDF5Matrix('test_8.h5', 'test_labels')

print(x_train.shape)
print(y_train.shape)
print(y_train[240])
print(x_test.shape)
print(y_test.shape)
    
model, callback_list = lstm()
#model.fit(x_train, y_train)
model.fit(x_train, y_train, batch_size = BATCH_SIZE, epochs = 100,verbose = 2,validation_data = (x_test, y_test),shuffle = 'batch', callbacks=callback_list)
    
model.save("Activity_Recognition.h5")

(703, 30, 2048)
(703, 9)
[0. 0. 0. 1. 0. 0. 0. 0. 0.]
(290, 30, 2048)
(290, 9)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 703 samples, validate on 290 samples
Epoch 1/100
 - 22s - loss: 2.5163 - acc: 0.1038 - top_k_categorical_accuracy: 0.5164 - val_loss: 2.2378 - val_acc: 0.1448 - val_top_k_categorical_accuracy: 0.5793
Epoch 2/100
 - 18s - loss: 2.4083 - acc: 0.1138 - top_k_categorical_accuracy: 0.5804 - val_loss: 2.1957 - val_acc: 0.1862 - val_top_k_categorical_accuracy: 0.6345
Epoch 3/100
 - 18s - loss: 2.3939 - acc: 0.1266 - top_k_categorical_accuracy: 0.5917 - val_loss: 2.1551 - val_acc: 0.2276 - val_top_k_categorical_accuracy: 0.6448
Epoch 4/100
 - 18s - loss: 2.3462 - acc: 0.1294 - top_k_categorical_accuracy: 0.5875 - val_loss: 2.1358 - val_acc: 0.2310 - val_top_k_categorical_accuracy: 0.6379
Epoch 5/100
 - 18s - loss: 2.3219 - acc: 0.1508 - top_k_cate

In [10]:
!ls

 Activity_Recognition.h5	   'models\checkpoint-50-1.71.hdf5'
 Human-Action-Classification-	   'models\checkpoint-51-1.71.hdf5'
 logs				   'models\checkpoint-52-1.70.hdf5'
'models\checkpoint-01-2.24.hdf5'   'models\checkpoint-53-1.71.hdf5'
'models\checkpoint-02-2.20.hdf5'   'models\checkpoint-54-1.70.hdf5'
'models\checkpoint-03-2.16.hdf5'   'models\checkpoint-55-1.70.hdf5'
'models\checkpoint-04-2.14.hdf5'   'models\checkpoint-56-1.70.hdf5'
'models\checkpoint-05-2.11.hdf5'   'models\checkpoint-57-1.68.hdf5'
'models\checkpoint-06-2.07.hdf5'   'models\checkpoint-58-1.69.hdf5'
'models\checkpoint-07-2.06.hdf5'   'models\checkpoint-59-1.68.hdf5'
'models\checkpoint-08-2.05.hdf5'   'models\checkpoint-60-1.70.hdf5'
'models\checkpoint-09-2.04.hdf5'   'models\checkpoint-61-1.69.hdf5'
'models\checkpoint-100-1.65.hdf5'  'models\checkpoint-62-1.69.hdf5'
'models\checkpoint-10-2.03.hdf5'   'models\checkpoint-63-1.70.hdf5'
'models\checkpoint-11-2.02.hdf5'   'models\checkpoint-64-1.69.hdf5'
'models\chec